In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [15]:
training_file = 'input.txt'
learning_rate = 0.001
training_iters = 40000
display_step = 1000
n_input = 3
n_hidden = 512


In [3]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
        
    content_final = []
    
    content = [str(x.strip()) for x in content]
    for i in range(len(content)):
        content_final += content[i].split()
    content = np.array(content_final)
    content = np.reshape(content, [-1, ])
    return content


In [4]:
training_data = read_data(training_file)

In [5]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [6]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [7]:
X = tf.placeholder(tf.float32,[None,n_input])
Y = tf.placeholder(tf.float32,[None,vocab_size])

w1 = tf.Variable(tf.random_normal([n_hidden, vocab_size]))
b1 = tf.Variable(tf.random_normal([vocab_size]))

def RNN(x):
    inp_sets = tf.split(x,n_input,axis=1)
    #Now define a 2-Layer LSTM , with each layer having n_hidden units
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    outputs, states = rnn.static_rnn(rnn_cell, inp_sets, dtype=tf.float32)
    # we'll take the last output and return it
    logit = tf.matmul(outputs[-1],w1) + b1
    return logit




In [8]:
pred = RNN(X)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))




In [16]:
session = tf.Session()
session.run(tf.global_variables_initializer())
offset = random.randint(0,n_input+1)
end_offset = n_input + 1
acc_total = 0
loss_total = 0

for step in range(training_iters):
    if offset > (len(training_data)-end_offset):
        offset = random.randint(0, n_input+1)
    
    # Token set to number set
    symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
    symbols_in_keys = np.reshape(np.array(symbols_in_keys), [1, n_input])
    symbols_out_onehot = np.zeros([vocab_size], dtype=float)
    symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
    symbols_out_onehot=np.reshape(symbols_out_onehot,[1,vocab_size])
    _, acc, cost, onehot_pred = session.run([optimizer, accuracy, loss, pred], \
                                                feed_dict={X: symbols_in_keys, Y: symbols_out_onehot})
    loss_total += cost
    acc_total += acc
    
    if (step+1) % display_step == 0:
        print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
        acc_total = 0
        loss_total = 0
        symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
        symbols_out = training_data[offset + n_input]
        symbols_out_pred = reverse_dictionary[int(session.run(tf.argmax(onehot_pred, 1)))]
        print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
    step += 1
    offset += (n_input+1)

print("Done training.....")

Iter= 1000, Average Loss= 4.192521, Average Accuracy= 8.10%
['we', 'could', 'easily'] - [escape] vs [escape]
Iter= 2000, Average Loss= 3.106584, Average Accuracy= 25.10%
['impossible', 'remedies', '.'] - [long] vs [nobody]
Iter= 3000, Average Loss= 2.217899, Average Accuracy= 43.20%
[',', 'therefore', ','] - [to] vs [to]
Iter= 4000, Average Loss= 1.809581, Average Accuracy= 56.50%
['wisest', 'little', 'pig'] - [had] vs [applause]
Iter= 5000, Average Loss= 1.387292, Average Accuracy= 63.30%
['signal', 'of', 'her'] - [approach] vs [approach]
Iter= 6000, Average Loss= 1.431020, Average Accuracy= 67.00%
['.', 'then', 'the'] - [old] vs [old]
Iter= 7000, Average Loss= 1.315205, Average Accuracy= 65.10%
[',', 'if', 'we'] - [could] vs [could]
Iter= 8000, Average Loss= 0.940569, Average Accuracy= 76.20%
['it', 'is', 'easy'] - [to] vs [to]
Iter= 9000, Average Loss= 1.037589, Average Accuracy= 74.20%
['treacherous', 'manner', 'in'] - [which] vs [agree]
Iter= 10000, Average Loss= 0.940790, Average

In [31]:

symbols_in = ['the','mice','take']
symbols_in = np.array([dictionary[token] for token in symbols_in])
for i in range(10000):
    
    symbols_in = np.reshape(symbols_in,[1,3])

    pred_out = session.run([pred],feed_dict={X:symbols_in})
    out_word = reverse_dictionary[int(np.argmax(np.squeeze(pred_out), 0))]
    print out_word
    symbols_in = np.reshape(symbols_in,[3])
    symbols_in = np.append(symbols_in[1:],int(np.argmax(np.squeeze(pred_out), 0)))

at
one
another
and
young
mouse
means
always
know
and
young
looked
common
one
in
the
neighbourhood
.
this
proposal
met
with
and
Luckily,
will
.
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
hous

house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
br

to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
th

treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
s

to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
th

the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly

house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
br

treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
scene
to
treacherous
fleeing
brothers.
house,
and
he
rapidly
opened
the
s

In [29]:
training_data.shape

(436,)

In [30]:
rnn

<module 'tensorflow.contrib.rnn' from '/Users/rahulchandran/tensorflow/lib/python2.7/site-packages/tensorflow/contrib/rnn/__init__.pyc'>